# Hash Table

* 一個資料庫裡就像是一個層櫃`table`，每一層之間都是互相獨立的空間，而我們將字串以HashMD5轉換成一串16進位的整數  
* 要怎麼將它分類呢？以取餘數的方式判定這個data會被放在哪一層櫃子`index`裡
* 但是一定會有`collision`的時候，代表有兩個data的餘數是一樣的，那這樣放進這層櫃子的話，data就會打架
* 為了解決這個問題，因此在每層抽屜裡，以鏈表(linked list）的方式去放入data
* 這樣的話就能實現O(1)的時間複雜度  
* array數組尋址容易、插入刪除困難，linkedlist鏈表尋址困難、插入刪除簡單，而Hash Table 可以實現兩者結合的優點：查詢、插入、刪除都簡單

![](https://i.imgur.com/yYNFE6X.png)  

# Hash Function 散列函數
* 又稱「哈希」函數
* 電子商務很常用到，為了雙重加密中的「消息摘要」
* 將不等長的資料轉換成為128位的編碼
* 我的hash table是以hash function裡的除法散列法 `index = int(code.hexdigest(),16) % capacity`
* 優點就是非常快，缺點則是`table`(capacity)要慎選
![](https://i.imgur.com/sF9KjCb.png)

>Reference  
>🔗[Hash Table：Intro(簡介)](http://alrightchiu.github.io/SecondRound/hash-tableintrojian-jie.html)  
>🔗[據說，80%的人都搞不懂哈希算法](https://kknews.cc/tech/jj2egxe.html)  
>🔗[十一、从头到尾解析Hash表算法](https://blog.csdn.net/v_JULY_v/article/details/6256463)

In [ ]:
#pip install crypto

In [1]:
from Crypto.Hash import MD5

### 定義MD5.new()

In [2]:
h = MD5.new()

### 轉成MD5

In [3]:
h.update("I am Yuni".encode("utf-8"))
print (h.hexdigest())

35cbe4ab5217190e0ec4d508e6d41f0b


### 轉成16進位

In [4]:
h = MD5.new("I am Yuni".encode("utf-8"))
int(h.hexdigest(),16)

71507758001694228210233983944688410379

In [5]:
from Crypto.Hash import MD5

class ListNode:
    def __init__(self,val):
        
        self.val = val
        self.next = None
        
class MyHashSet:
    
    def __init__(self, capacity=5):
        """
        :type capacity: int
        :rtype: None
        """
        self.capacity = capacity      #定義抽屜（有幾個table）
        self.data = [None] * capacity #整張桌子有幾個抽屜，一開始所有抽屜裡面都是None
        
        #capacity = 5，self.data = [None],[None],[None],[None],[None]
        #self.data[index] -> if index = 0 ->會是第0個[None]，index總共有0~4
        #所以假設self.data是個櫃子，index是第幾層抽屜，抽屜與抽屜間沒有任何關係
        
    def convert(self,key):
        
        code = MD5.new()
        code.update(key.encode("utf-8"))
        code.hexdigest()
        int(code.hexdigest(),16)       #將轉換成MD5的散列換算成16進位的整數（int）
        return int(code.hexdigest(),16)
        
    def add(self, key):
        """
        :type key: str
        :rtype: None
        """
        count = self.convert(key)       #先將key（str）轉換成整數
        index = count % self.capacity   #決定要將key放在哪個index(第幾層抽屜)上，取餘數
        
        if self.data[index] is None:
            self.data[index] = ListNode(count) #假設這個抽屜裡沒有東西，就將key -> 整數，放進對應的抽屜
            
        else:    #那如果今天抽屜裡已經有東西了，此狀況稱為collision(碰撞)，我的方法是讓他成為抽屜裡原本已存在的最後一位
            
            head = self.data[index]  #設self.data[index]為頭
            
            while head.next is not None: #如果head下一位是還有節點的，設下一位為head，直到走訪到head沒有指向為止
                head = head.next
                
            head.next = ListNode(count) #放在抽屜的最後一個位置


    def remove(self, key):
        """
        :type key: str
        :rtype: None
        """      
        count = self.convert(key) #先將key轉換
        
        if self.contains(key): #如果key有存在這個table裡的話，move on 
            
            index = count % self.capacity 
            
            if self.data[index].val == count:    #如果那個抽屜的值==要找的key(此時已轉換為count)
                self.data[index]= self.data[index].next   
                
                #那麼這個抽屜中第一個節點將被下一個節點取代，如果抽屜只有一個val，代表self.data.next = None
                #移除self.data[index]後，self.data[index].next = None將會取代self.data[index]
            
            else: #如果self.data[index]不是要找的key的話，走訪這個抽屜內的其他node
                head = self.data[index]    #設抽屜中第一個為head
                while head.next != count:  #當head的下一位不是要找的key，move on 
                    head = head.next       #讓head指向下一位的節點成為新的head，直到找到head.next == count
                head.next = head.next.next #因為要移除head.next，所以head.next.next將會成為新的head.next
                     
        if self.contains(key):
            self.remove(key)   #如果抽屜裡有兩個相同的值要一起移除
        return 
                
    def contains(self, key):
        """
        :type key: str
        :rtype: bool(True or False)
        """
        count = self.convert(key)
        index = count % self.capacity
        
        if self.data[index] is None:     #如果self.data[index] = None，代表這個抽屜沒有東西，回傳False
            return False
        else:  #若是要找的key的抽屜有東西的話，要找，因為不一定self.data[index]就是要找的key
            head = self.data[index]     
            while head is not None and head.val != count: #當head不是None且head.val不是key時，head.next成為新的head
                head = head.next
            if head:        
                return True
            else:
                return False
            #如果head存在（代表head.val == count，或是while的條件不成立，代表著self.data[index].val == count)，return True，不存在則False

In [6]:
hashSet = MyHashSet()
hashSet.add("dog")
hashSet.add("pig")
rel = hashSet.contains("pig")
print(rel)
rel = hashSet.contains("dog")
print(rel)
rel = hashSet.contains("cat")
print(rel)
hashSet.add("bird")
rel = hashSet.contains("bird")
print(rel)

True
True
False
True


In [7]:
hashSet.remove("pig")

In [8]:
hashSet.contains("pig")

False

In [9]:
hashSet.add("dog")

In [10]:
hashSet.remove("dog")

In [11]:
hashSet.contains("dog")

False

### Hash Table流程圖
![](https://i.imgur.com/9LuIYDm.png)

In [12]:
from Crypto.Hash import MD5

class ListNode:
    
    def __init__(self,val):
        
        self.val = val
        self.next = None
       
    
class MyHashSet:
    
    def __init__(self, capacity=5):
        """
        :type capacity: int
        :rtype: None
        """
        self.capacity = capacity
        self.data = [None] * capacity
        
    def convert(self,key):
        
        code = MD5.new()
        code.update(key.encode("utf-8"))
        code.hexdigest()
        int(code.hexdigest(),16)
        return int(code.hexdigest(),16)
        
    def add(self, key):
        """
        :type key: str
        :rtype: None
        """
        count = self.convert(key)
        index = count % self.capacity
        
        node = ListNode(key)
        
        if self.data[index] is None:
            self.data[index] = node
            
        else:
            
            head = self.data[index]
            
            while head.next is not None:
                head = head.next
                
            head.next = node


    def remove(self, key):
        """
        :type key: str
        :rtype: None
        """
        count = self.convert(key)
        
        if self.contains(key):
            index = count % self.capacity
            
            if self.data[index] and self.data[index].val == count:
                self.data[index] = self.data[index].next
            #self.data[index]為要移除的key，所以當self.data[index] == count的時候，將下一位直接往前一位
                
            pre = None     #使用previous的方式，先設pre為None
            while self.data[index]: #當self.data[index] == True，move on 
                if self.data[index].val == count:  #如果找到了要移除的key，那麼self.data[index].next會變成pre.next
                    pre.next = self.data[index].next
                    return
                pre = self.data[index]  #如果第一圈找self.data[index]沒找到，那麼會將self.data[index]變成pre
                self.data[index] = self.data[index].next #而self.data[index].next會變成self.data[index]
                           
    def contains(self, key):
        """
        :type key: str
        :rtype: bool(True or False)
        """
        count = self.convert(key)
        index = count % self.capacity
        
        if self.data[index] is None:
            return False
        else:
            head = self.data[index]
            while head is not None and head.val != key:
                head = head.next
            if head:
                return True
            else:
                return False    

In [13]:
a = MyHashSet()
a.add("animal")
a.add("ocean")
a.add("pig")
a.add("duck")
a.add("dog")
a.add("tiger")
print(a.contains("animal"))
a.remove("duck")
print(a.contains("duck"))
a.remove("pig")
print(a.contains("pig"))
a.remove("tiger")
print(a.contains("tiger"))

True
False
False
False
